In [11]:
import pandas as pd

In [12]:
merged_data_2007_to_2012 = pd.read_csv("/data/workspace_files/lazy_price_replication/10k_final_with_ticker_name_filtered_w_similarity_and_sentiment.csv", index_col=0)

In [13]:
len(merged_data_2007_to_2012)

725

In [14]:
merged_data_2007_to_2012.head()

,cik,report_period_end_date2007,file_date2007,statement2007,report_period_end_date2008,file_date2008,statement2008,report_period_end_date2009,file_date2009,statement2009,...,jaccard_similarity_2008_to_2009,jaccard_similarity_2009_to_2010,jaccard_similarity_2010_to_2011,jaccard_similarity_2011_to_2012,min_edit_distance_similarity_2007_to_2008,min_edit_distance_similarity_2008_to_2009,min_edit_distance_similarity_2009_to_2010,min_edit_distance_similarity_2010_to_2011,min_edit_distance_similarity_2011_to_2012,sentiment2007
0,1800,20071231,20080219,ITEM 7. MANAGEMENTS DISCUSSION AND ANALYSIS OF...,20081231,20090220,ITEM 7. MANAGEMENTS DISCUSSION AND ANALYSIS OF...,20091231,20100219,ITEM 7. MANAGEMENTS DISCUSSION AND ANALYSIS OF...,...,0.774971,0.693548,0.701555,0.755847,18063,18430,29949,40821,33152,0.121238
2,2178,20071231,20080331,Item 7. MANAGEMENTS DISCUSSION AND ANALYSIS OF...,20081231,20090324,Item 7. MANAGEMENTS DISCUSSION AND ANALYSIS OF...,20091231,20100323,Item 7. MANAGEMENTS DISCUSSION AND ANALYSIS OF...,...,0.679785,0.685680,0.694479,0.741176,19732,16025,15412,11842,10627,0.350000
3,3453,20071231,20080228,ITEM 7. MANAGEMENTS DISCUSSION AND ANALYSIS OF...,20081231,20090227,ITEM 7. MANAGEMENTS DISCUSSION AND ANALYSIS OF...,20091231,20100226,ITEM 7. MANAGEMENTS DISCUSSION AND ANALYSIS OF...,...,0.734218,0.750826,0.725908,0.573413,47545,37163,35024,29475,46760,0.200000
4,3499,20071231,20080225,Item 7. Managements Discussion and Analysis of...,20081231,20090223,Item 7. Managements Discussion and Analysis of...,20091231,20100222,Item 7. Managements Discussion and Analysis of...,...,0.662145,0.733200,0.693905,0.710875,14165,21125,18948,18335,13204,0.030000
5,3545,20070831,20071114,Item 7. Managements Discussion and Analysis of...,20080930,20081215,Item 7. Managements Discussion and Analysis of...,20090930,20091214,Item 7. Managements Discussion and Analysis of...,...,0.659204,0.681324,0.484427,0.442766,35463,26189,30443,73158,73257,-0.150000


## Data Cleaning and Preparation

### Section to generate sentiment score from gpt

In [16]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from typing import List, Tuple, Dict, Any
import requests
import http
import json
from time import sleep
from tqdm import tqdm
from time import monotonic
import tiktoken

In [17]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/datalore/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [18]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/datalore/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [19]:
gpt_model: str = "gpt-3.5-turbo"
token_limit_per_min: int = 45000
token_limit_per_request: int = 13200
orgnization_token_limit_per_min: int = 80000

In [20]:
gpt_token = ''

In [21]:
def split_paragraph_into_chunks(paragraph: str, token_limit: int)->List[Tuple[str, float]]:
    """
    Because GPT has limit on how many tokens we can send in each request, we need to
    breakdown a long paragraph into smaller chunks based on sentences

    @param paragraph The long paragraph to be split
    @param token_limit The token limit for each chunk. Based on GPT documentation, each token
    is approximately 0.75 word

    @returns a list of broken down chunks in tuple format. The first element in the tuple is
    the actual text, and the second element is the estimated tokens used by this text
    """
    # Split paragraph into sentences
    sentences = sent_tokenize(paragraph)
    
    chunks = []
    current_chunk = ""
    current_word_count = 0
    
    for sentence in sentences:
        sentence_word_count = len(word_tokenize(sentence))
        # print(f"got {sentence_word_count=}")
        assert sentence_word_count/0.75 <= token_limit, f"cannot handle long sentence with word count: {sentence_word_count}, exceeding {token_limit=} since {sentence_word_count/0.75=}"
        # If adding the current sentence exceeds the word limit, start a new chunk
        if (current_word_count + sentence_word_count)/0.75 > token_limit:
            chunks.append((current_chunk.strip(), current_word_count/0.75))
            current_chunk = sentence
            current_word_count = sentence_word_count
            # print(f"appending to chunks, currently {len(chunks)=}")
        else:
            # Add the sentence to the current chunk
            current_chunk += " " + sentence
            current_word_count += sentence_word_count
            # print(f"appending to current_chunk {len(current_chunk)=}, {current_word_count=}")
    
    if current_chunk:
        chunks.append((current_chunk.strip(), current_word_count/0.75))
    
    return chunks

In [22]:
def split_paragraph_into_chunks_v2(paragraph: str, token_limit: int)->List[Tuple[str, float]]:
    """
    Because GPT has limit on how many tokens we can send in each request, we need to
    breakdown a long paragraph into smaller chunks based on sentences

    @param paragraph The long paragraph to be split
    @param token_limit The token limit for each chunk. Based on GPT documentation, each token
    is approximately 0.75 word

    @returns a list of broken down chunks in tuple format. The first element in the tuple is
    the actual text, and the second element is the estimated tokens used by this text
    """
    # Split paragraph into sentences
    sentences = sent_tokenize(paragraph)
    
    chunks = []
    current_chunk = ""
    current_word_count = 0
    
    for sentence in sentences:
        sentence_word_count = len(word_tokenize(sentence))
        # print(f"got {sentence_word_count=}")
        assert sentence_word_count/0.75 <= token_limit, f"cannot handle long sentence with word count: {sentence_word_count}, exceeding {token_limit=} since {sentence_word_count/0.75=}"
        # If adding the current sentence exceeds the word limit, start a new chunk
        if (current_word_count + sentence_word_count)/0.75 > token_limit:
            chunks.append((current_chunk.strip(), current_word_count/0.75))
            current_chunk = sentence
            current_word_count = sentence_word_count
            # print(f"appending to chunks, currently {len(chunks)=}")
        else:
            # Add the sentence to the current chunk
            current_chunk += " " + sentence
            current_word_count += sentence_word_count
            # print(f"appending to current_chunk {len(current_chunk)=}, {current_word_count=}")
    
    if current_chunk:
        chunks.append((current_chunk.strip(), current_word_count/0.75))
    
    return chunks

In [23]:
def build_gpt_messages(text_content: str, prompt_text: str)->Tuple[List[List[Dict[str, str]]], List[float]]:
    """
    Given a text input, internally breaks it down based on GPT token limits 
    (for https://platform.openai.com/docs/models/gpt-3-5-turbo model, it is 16,385 tokens, setting
     it to 15000 to leave some buffer)
    
    @param text_content The input text
    @param prompt_text The prompt text for GPT

    @returns a tuple whose element is a list of conversation with GPT, each conversation is a broken
    down portion of original input text based on GPT token limit. The second element is simply a list
    of estimated used tokens for each conversation
    """
    chunks = split_paragraph_into_chunks(text_content, token_limit_per_request)
    # print(f"{len(chunks)=}")
    input_messages = [[{"role": "system", "content": prompt_text}, {"role": "user", "content": chunk[0]}] for chunk in chunks]
    estimated_tokens = [chunk[1] for chunk in chunks]
    return input_messages, estimated_tokens

In [24]:
def send_chat_completion_post(prompt_message: Dict[str, str], 
                              user_message:Dict[str, str], 
                              is_response_json: bool = False) -> Dict[str, Any]:
    gpt_url = "https://api.openai.com/v1/chat/completions"
    headers = {"Content-Type": "application/json", "Authorization": "Bearer {}".format(gpt_token)}
    payload = {"model": gpt_model, "messages": [prompt_message, user_message]}
    if is_response_json:
        payload.update({"response_format": {"type": "json_object"}})

    # try requesting with increasing number of timeout
    response = None
    timeouts = (5, 10, 20, 40)
    for timeout in timeouts:
        try:
            response = requests.post(gpt_url, headers=headers, data=json.JSONEncoder().encode(payload), timeout=timeout)
            # print(f"{response.json()=}")
        except requests.exceptions.Timeout:
            print("Request timed out after {:0d} seconds, trying again...".format(timeout))
        except requests.exceptions.RequestException as r_exc:
            print("Request error:\n", str(r_exc))
        except Exception as exc:
            print("Unknown exception:\n", str(exc))
        else:
            if response:
                break
            else:
                print("Request empty, trying again...")
                sleep(40)

    # check if have response and only extract response object if status is OK
    if response:
        if response.status_code == http.HTTPStatus.OK:
            return response.json()
    print(f"got error response: {response} {response.json()}")
    return {}

In [25]:
prompt = "You will be provided with a portion of articles. Help me calculate a sentiment score in a separate json field called sentiment_score between -1 and 1 where -1 is most negative and 1 is most positive."

In [26]:
def enrich():
    cur_tokens_left = token_limit_per_min
    replenish_rate_per_sec = token_limit_per_min / 60

    for year in range(2008, 2013):
        print(f"processing statements for {year}...")
        sentiment_for_this_year = []
        for index, row in tqdm(merged_data_2007_to_2012.iterrows(), total=merged_data_2007_to_2012.shape[0]):
            # break down long text into chunks
            statement_text = row[f"statement{year}"]
            inputs = build_gpt_messages(statement_text, prompt)
            start_time = monotonic()
            responses_raw = []
            # process chunk by chunk and store them into the list
            for conversation, estimated_tokens_needed in zip(inputs[0], inputs[1]):
                if cur_tokens_left - estimated_tokens_needed < 0:
                    sleep_time = int((estimated_tokens_needed - cur_tokens_left)/replenish_rate_per_sec + 1)
                    print(f"sleeping for {sleep_time} seconds")
                    sleep(sleep_time)
                now = monotonic()
                cur_tokens_left = (now-start_time)*replenish_rate_per_sec + cur_tokens_left
                assert cur_tokens_left >= estimated_tokens_needed, f"insufficient {cur_tokens_left=}, {estimated_tokens_needed=}"
                retry = 0
                while retry <= 3:
                    response = send_chat_completion_post(conversation[0], conversation[1], True)
                    if len(response) == 0:
                        print(f"got empty response. There is probably some errors. Need to sleep and retry")
                        more_sleep_time = int(orgnization_token_limit_per_min/replenish_rate_per_sec + 1)
                        print(f"sleeping for {more_sleep_time} seconds to fully replenish tokens...")
                        sleep(more_sleep_time)
                        retry += 1
                        cur_tokens_left = orgnization_token_limit_per_min - 10000
                        continue
                    # print(f"got response: {response}")
                    cur_tokens_left -= int(response['usage']['total_tokens'])
                    responses_raw.append(response)
                    break
                if retry > 3:
                    print(f"got empty response. after {retry} retries. Need to manual inspect what is going wrong...")
                    return
            
            # parse the individual response, and aggregate the result
            scores = []
            num_tokens = []
            for idx, res in enumerate(responses_raw):
                # print(f"got content: {res['choices'][0]['message']['content']}")
                try:
                    parsed_res = json.loads(res['choices'][0]['message']['content'])
                    score = float(parsed_res['sentiment_score'])
                    scores.append(score)
                    num_tokens.append(res['usage']['prompt_tokens'])
                    # print(f"estimated tokens: {inputs[1][idx]}, actual used: {res['usage']['prompt_tokens']}, diff: {inputs[1][idx] - res['usage']['prompt_tokens']}")
                except Exception as e:
                    print(f"encountered exception {e} while processing index {index} for year {year}")
            
            if len(scores) > 0:
                total_tokens = sum(num_tokens)
                avg_score = 0.0
                for idx, original_score in enumerate(scores):
                    avg_score += original_score * (num_tokens[idx]/total_tokens)
                sentiment_for_this_year.append(avg_score)
            else:
                print(f"encountered empty scores list --- {index} --- setting it to 0")
                sentiment_for_this_year.append(0)
        print(f"setting scores for {year}...")
        merged_data_2007_to_2012[f"sentiment{year}"] = sentiment_for_this_year
        new_filename = "/data/workspace_files/lazy_price_replication/10k_final_with_ticker_name_filtered_w_similarity_and_sentiment.csv"
        print(f"saving files to {new_filename}")
        merged_data_2007_to_2012.to_csv(new_filename)

In [27]:
enrich()

processing statements for 2008...
Request timed out after 5 seconds, trying again...
sleeping for 5 seconds
sleeping for 17 seconds
sleeping for 15 seconds
sleeping for 5 seconds
sleeping for 15 seconds
sleeping for 14 seconds
sleeping for 10 seconds
sleeping for 16 seconds
sleeping for 4 seconds
sleeping for 6 seconds
sleeping for 11 seconds
sleeping for 9 seconds
sleeping for 12 seconds
sleeping for 15 seconds
sleeping for 2 seconds
sleeping for 1 seconds
sleeping for 3 seconds
sleeping for 15 seconds
sleeping for 4 seconds
sleeping for 1 seconds
sleeping for 14 seconds
sleeping for 10 seconds
sleeping for 17 seconds
sleeping for 14 seconds
sleeping for 8 seconds
sleeping for 15 seconds
sleeping for 10 seconds
sleeping for 7 seconds
sleeping for 4 seconds
sleeping for 18 seconds
sleeping for 5 seconds
sleeping for 1 seconds
sleeping for 15 seconds
sleeping for 16 seconds
sleeping for 14 seconds
sleeping for 8 seconds
sleeping for 1 seconds
sleeping for 4 seconds
sleeping for 15 secon

  1%|          | 6/725 [01:52<3:45:03, 18.78s/it]


KeyboardInterrupt: KeyboardInterrupt

In [39]:
merged_data_2007_to_2012.head()

,cik,report_period_end_date2007,file_date2007,statement2007,report_period_end_date2008,file_date2008,statement2008,report_period_end_date2009,file_date2009,statement2009,...,jaccard_similarity_2008_to_2009,jaccard_similarity_2009_to_2010,jaccard_similarity_2010_to_2011,jaccard_similarity_2011_to_2012,min_edit_distance_similarity_2007_to_2008,min_edit_distance_similarity_2008_to_2009,min_edit_distance_similarity_2009_to_2010,min_edit_distance_similarity_2010_to_2011,min_edit_distance_similarity_2011_to_2012,sentiment2007
0,1800,20071231,20080219,ITEM 7. MANAGEMENTS DISCUSSION AND ANALYSIS OF...,20081231,20090220,ITEM 7. MANAGEMENTS DISCUSSION AND ANALYSIS OF...,20091231,20100219,ITEM 7. MANAGEMENTS DISCUSSION AND ANALYSIS OF...,...,0.774971,0.693548,0.701555,0.755847,18063,18430,29949,40821,33152,0.121238
2,2178,20071231,20080331,Item 7. MANAGEMENTS DISCUSSION AND ANALYSIS OF...,20081231,20090324,Item 7. MANAGEMENTS DISCUSSION AND ANALYSIS OF...,20091231,20100323,Item 7. MANAGEMENTS DISCUSSION AND ANALYSIS OF...,...,0.679785,0.685680,0.694479,0.741176,19732,16025,15412,11842,10627,0.350000
3,3453,20071231,20080228,ITEM 7. MANAGEMENTS DISCUSSION AND ANALYSIS OF...,20081231,20090227,ITEM 7. MANAGEMENTS DISCUSSION AND ANALYSIS OF...,20091231,20100226,ITEM 7. MANAGEMENTS DISCUSSION AND ANALYSIS OF...,...,0.734218,0.750826,0.725908,0.573413,47545,37163,35024,29475,46760,0.200000
4,3499,20071231,20080225,Item 7. Managements Discussion and Analysis of...,20081231,20090223,Item 7. Managements Discussion and Analysis of...,20091231,20100222,Item 7. Managements Discussion and Analysis of...,...,0.662145,0.733200,0.693905,0.710875,14165,21125,18948,18335,13204,0.030000
5,3545,20070831,20071114,Item 7. Managements Discussion and Analysis of...,20080930,20081215,Item 7. Managements Discussion and Analysis of...,20090930,20091214,Item 7. Managements Discussion and Analysis of...,...,0.659204,0.681324,0.484427,0.442766,35463,26189,30443,73158,73257,-0.150000


### Download adjusted close from yahoo finance

In [10]:
import yfinance as yf
from tqdm import tqdm
import time

In [14]:
def get_ticker_prices(start_date: str = "2008-01-01", end_date="2013-12-31")->None:
    failed_downloads = []
    for ticker in tqdm(merged_data_2007_to_2012.ticker.unique()):
        try:
            data = yf.download(ticker, start=start_date, end=end_date, progress=False)
            data.to_csv(f"/data/workspace_files/lazy_price_replication/ticker_prices_by_years/{ticker}.csv")
        except Exception as e:
            print(f"failed download for {ticker} with {e}")
        time.sleep(0.1)
    print(f"total failed downloads: {len(failed_downloads)}: {failed_downloads}")

In [15]:
get_ticker_prices()


1 Failed download:
- ACET: Data doesn't exist for startDate = 1199163600, endDate = 1388466000

1 Failed download:
- CHEX: No data found for this date range, symbol may be delisted

1 Failed download:
- IDSA: No data found for this date range, symbol may be delisted

1 Failed download:
- ALGI: No data found for this date range, symbol may be delisted

1 Failed download:
- AVP: No data found for this date range, symbol may be delisted
failed download for nan with 'float' object has no attribute 'replace'

1 Failed download:
- BGG: No data found for this date range, symbol may be delisted

1 Failed download:
- CDI: No data found for this date range, symbol may be delisted

1 Failed download:
- CTL: No data found for this date range, symbol may be delisted

1 Failed download:
- CHRS: Data doesn't exist for startDate = 1199163600, endDate = 1388466000

1 Failed download:
- CSS: No data found for this date range, symbol may be delisted

1 Failed download:
- ABC: No data found for this date

100%|██████████| 1182/1182 [06:03<00:00,  3.25it/s]


In [16]:
from os import listdir
from os.path import isfile, join
from typing import List

In [25]:
def merge_ticker_prices()->List[str]:
    folder_path = "/data/workspace_files/lazy_price_replication/ticker_prices_by_years/"
    ticker_price_csv = [f for f in listdir(folder_path) if isfile(join(folder_path, f))]
    empty_data_ticker_name = []
    normal_data = []
    csv_suffix = '.csv'
    for csv_file in tqdm(ticker_price_csv):
        ticker_name = csv_file[:-len(csv_suffix)]
        full_path = join(folder_path, csv_file)
        data = pd.read_csv(full_path)
        if len(data) == 0:
            empty_data_ticker_name.append(ticker_name)
        else:
            normal_data.append(data)
            data["ticker"] = ticker_name
    merged_prices = pd.concat(normal_data)
    merged_prices.to_csv("/data/workspace_files/lazy_price_replication/all_ticker_prices.csv")
    return empty_data_ticker_name


In [26]:
tickers_with_empty_df = merge_ticker_prices()

100%|██████████| 1181/1181 [00:06<00:00, 190.27it/s]


In [27]:
len(tickers_with_empty_df)

488

### Instructions:

Here is the original [github link](https://github.com/michaelewens/MD-A-10-K-data/tree/master?tab=readme-ov-file). The example code is demonstrating how to parse the original 10-K based on keywords. For us, we don't care about those keywords and want to process the entire section. This is easy for the already scraped dataset [here](https://data.caltech.edu/records/28m72-ysc35). But this dataset only contains the data from 2002-2018. If we have more time, we could use [this github](https://github.com/apodobytko/10K-MDA-Section) to extract 10-K reports for more recent years.

NOTE: for the parsed pandas dataframe, the CIK number for the company is a unique ID issued by SEC, and the SIC number is the industry code which it belongs to. The data is then joined with the `cik_to_cusip_wrds.csv` file, which has the mapping from cik to cusip number.

NOTE: [WRDS](https://wrds-www.wharton.upenn.edu/pages/get-data/wrds-sec-analytics-suite/wrds-sec-linking-tables/cik-cusip-link-table/) has a better way of mapping cik to cusip. Once we have parsed all 10-K file, we can then do a left join with this dataset.

The overall data preparation flow is:
1. parse 10k data in yearly fashion (`10k_parsed_by_years`)
2. link 10k data to cusip based on cik
3. filter the linked data by only the cusip used in #2 

In [1]:
import pandas as pd
import os
from tqdm import tqdm
import re
from typing import List, Any, Optional, Tuple
from zipfile import ZipFile
from zipfile import ZipInfo
from dataclasses import dataclass
import concurrent.futures

### Section to extract mda section

In [ ]:
@dataclass
class ParseResult:
    parsed_df: Optional[pd.DataFrame] = None
    total_skipeed: int = 0

In [ ]:
def parse_10k_mda_section(text:str)->ParseResult:
    pattern = r'<HEADER>(.*?)</HEADER>\s*<SECTION>(.*?)</SECTION>'
    parsed_df = pd.DataFrame(columns=["company_name", "cik", "sic", "form_type", "report_period_end_date", "file_date", "content"])

    # Find all matches using the regex pattern
    matches = re.findall(pattern, text, re.DOTALL)
    # Extract header text and section text pairs
    # NOTE: each file might contain multiple 10K for different companies
    current_row = {}
    total_skipped = 0
    for match in matches:
        header_text = match[0].strip()
        # header text contains information like this:
        # '\nCOMPANY NAME: PINNACLE WEST CAPITAL CORP\nCIK: 0000764622\nSIC: 4911\nFORM TYPE: 10-K\nREPORT PERIOD END DATE: 20151231\nFILE DATE: 20160219\n'
        header_text_split = header_text.split("\n")
        if len(header_text_split) != 6:
            total_skipped += 1
            continue
        should_skip = False
        for header_info in header_text_split:
            if not should_skip:
                header_tokens = header_info.split(":")
                if len(header_tokens) == 2:
                    header_type = header_tokens[0].strip()
                    header_value = header_tokens[1].strip()
                    if header_type == "COMPANY NAME":
                        current_row["company_name"] = header_value
                    elif header_type == "CIK":
                        current_row["cik"] = header_value
                    elif header_type == "SIC":
                        current_row["sic"] = header_value
                    elif header_type == "FORM TYPE":
                        current_row["form_type"] = header_value
                    elif header_type == "REPORT PERIOD END DATE":
                        current_row["report_period_end_date"] = header_value
                    elif header_type == "FILE DATE":
                        current_row["file_date"] = header_value
                    else:
                        should_skip = True
                else:
                    should_skip = True
        if should_skip:
            total_skipped += 1
            continue

        current_row["content"] = match[1].strip()
        parsed_df = pd.concat([parsed_df, pd.DataFrame(current_row, index=[0])], ignore_index=True)
    return ParseResult(parsed_df if len(parsed_df)>0 else None, total_skipped)

In [ ]:
def divide_list_into_portions(lst: List[Any], num_portions: int)->List[List[Any]]:
    avg_len = len(lst) / num_portions
    portions = []
    last = 0.0

    while last < len(lst):
        portions.append(lst[int(last):int(last + avg_len)])
        last += avg_len

    return portions

In [ ]:
def extract_mda_inner(portion: List[ZipInfo], dest_dir: str, opened_zip: ZipFile, log_prefix: str)->int:
    total_skipped_in_cur_portion = 0
    processed = 0
    for info in portion:
        if not info.filename.startswith("__MACOSX"):
            try:
                file_name = info.filename.split('/')[-1] 
                file_id = file_name.split('.')[0]
                if len(file_id) > 0:
                    with opened_zip.open(info.filename, "r") as cur_file:
                        cur_text = cur_file.read().decode('utf-8')
                    parsed = parse_10k_mda_section(cur_text)
                    total_skipped_in_cur_portion += parsed.total_skipeed
                    if parsed.parsed_df is not None:
                        parsed.parsed_df.to_csv(f"{os.path.join(dest_dir, file_id)}.csv")
            except Exception as e:
                print(f"skipping file: {info.filename} due to error: {e}")
        processed += 1
        if processed % 200 == 0:
            print(f"{log_prefix} processed: {processed}/{len(portion)}..")
    return total_skipped_in_cur_portion

In [ ]:
def extract_mda(source_dir: str, source_filename: str, dest_dir: str) -> None:
    full_path = os.path.join(source_dir, source_filename)
    print(f"processing: {full_path} with {source_dir=}, {source_filename=}, {dest_dir=}...")
    total_skipped_in_zip = 0
    with ZipFile(full_path, 'r') as cur_zip_file:
        print(f"opened zip file {full_path=}")
        info_list = cur_zip_file.infolist()
        print(f"{source_filename} got {len(info_list)} info list")
        total_skipped_in_cur_portion = 0
        processed = 0
        for info in info_list:
            if not info.filename.startswith("__MACOSX"):
                try:
                    file_name = info.filename.split('/')[-1] 
                    file_id = file_name.split('.')[0]
                    if len(file_id) > 0:
                        with cur_zip_file.open(info.filename, "r") as cur_file:
                            cur_text = cur_file.read().decode('utf-8')
                        parsed = parse_10k_mda_section(cur_text)
                        total_skipped_in_cur_portion += parsed.total_skipeed
                        if parsed.parsed_df is not None:
                            parsed.parsed_df.to_csv(f"{os.path.join(dest_dir, file_id)}.csv")
                except Exception as e:
                    print(f"skipping file: {info.filename} due to error: {e}")
            processed += 1
            if processed % 1000 == 0 or processed == 1:
                print(f"{source_filename} processed: {processed}/{len(info_list)}..")
        
    print(f"total skipped parsed results: {total_skipped_in_zip}")

In [ ]:
def ensure_filenames_unique(source_dir: str, source_filename: str)->None:
    full_path = os.path.join(source_dir, source_filename)
    print(f"processing: {full_path}...")
    with ZipFile(full_path, 'r') as cur_zip_file:
        info_list = cur_zip_file.infolist()
        seen = set()
        for info in tqdm(info_list):
            if not info.filename.startswith("__MACOSX"):
                file_name = info.filename.split('/')[-1] 
                file_id = file_name.split('.')[0]
                if len(file_id) > 0:
                    if file_id in seen:
                        print(f"found duplicate file name: {file_name}")
                    seen.add(file_id)

In [ ]:
def parse_all_files_in_parallel(inputs_to_parse: List[Tuple[str, str, str]]) -> None:
    parse_file_futures = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
        for source_dir, source_filename, dest_dir in inputs_to_parse:
            parse_file_futures.append(executor.submit(extract_mda, source_dir, source_filename, dest_dir))

        for i, f in enumerate(parse_file_futures):
            try:
                f.result()
                print(f"done parsing file: {inputs_to_parse[i][1]}..")
            except Exception as e:
                print(f"failed to parse file: {inputs_to_parse[i][1]}..{e}")

In [ ]:
def merge_parsed_dfs()->None:
    top_dir = '/data/workspace_files/10k_parsed'
    dir_list = os.listdir(top_dir)
    small_files_to_merge: List[str] = []
    for sub_dir in dir_list:
        sub_dir_full_path = os.path.join(top_dir, sub_dir)
        files_in_sub_dir = os.listdir(sub_dir_full_path)
        for filename in files_in_sub_dir:
            small_files_to_merge.append(os.path.join(sub_dir_full_path, filename))
    print(f"Total files: {len(small_files_to_merge)} in {len(dir_list)} folders..")
    num_merged_files = int(10/0.5)
    files_to_merge_divided = divide_list_into_portions(small_files_to_merge, num_merged_files)
    file_lengths = [len(lst) for lst in files_to_merge_divided]
    total_files_sum = sum(file_lengths)
    print(f"expected {len(small_files_to_merge)}, actual: {total_files_sum}, with lengths: {file_lengths}")
    for i, lst in enumerate(files_to_merge_divided):
        print(f"processing {i}/{len(files_to_merge_divided)} file list...")
        df_merged = None
        for filename in tqdm(lst):
            df_small = pd.read_csv(filename)
            if df_merged is None:
                df_merged = df_small
            else:
                df_merged = pd.concat([df_merged, df_small])
        if df_merged is not None:
            df_merged.to_csv(f"/data/workspace_files/lazy_price_replication/10k_parsed_merged/merged_{i}.csv")

In [ ]:
def breakdown_by_report_period_end_date():
    year_to_start_end = {
        year: (year*10000+101, year*10000+1231) for year in range(2002, 2019)
    }
    # print(year_to_start_end)
    dirname = "/data/workspace_files/10k_parsed_merged"
    merged_files = os.listdir(dirname)
    for year, start_end in year_to_start_end.items():
        # print(f"processing {year=}, {start_end=}")
        df_by_year = None
        start, end = start_end
        for filename in tqdm(merged_files):
            full_filename = os.path.join(dirname, filename)
            # print(f"processing {full_filename}")
            df = pd.read_csv(full_filename)
            # print(f"done reading {full_filename}")
            df_filtered = df[(df['report_period_end_date'] >= start) & (df['report_period_end_date'] <= end)]
            # print(f"done filtering {full_filename}")
            if df_by_year is None:
                df_by_year = df_filtered
            else:
                df_by_year = pd.concat([df_by_year, df_filtered])
        if df_by_year is None:
            print(f"!!!! this is unlikely but we found None for year {year}")
        else:
            df_by_year.to_csv(f"/data/workspace_files/lazy_price_replication/10k_parsed_by_years/{year}.csv")
        print(f"done processing file for {year}..")

### Some Data Cleaning

In [2]:
# NOTE: do not execute the code below again

# current_processing_year = 2012
# filename_to_dup = f"/data/workspace_files/lazy_price_replication/10k_parsed_by_years/{current_processing_year}.csv"
# print(f"reading csv from {filename_to_dup}")
# df_with_dup_columns = pd.read_csv(filename_to_dup)
# df_with_dup_columns.head()
# df_with_dup_columns = df_with_dup_columns[["company_name", "cik", "sic", "form_type", "report_period_end_date", "file_date", "content"]]
# df_with_dup_columns = df_with_dup_columns.rename(columns={"content": f"{current_processing_year}_statement",})
# df_with_dup_columns.head()
# print(f"saving csv tp {filename_to_dup}")
# df_with_dup_columns.to_csv(filename_to_dup)
# join_year_left = 2011
# join_year_right = 2012
# left_df = pd.read_csv(f"/data/workspace_files/lazy_price_replication/10k_parsed_by_years/{join_year_left}.csv")
# right_df = pd.read_csv(f"/data/workspace_files/lazy_price_replication/10k_parsed_by_years/{join_year_right}.csv")
# merged_df = pd.merge(left_df, right_df, on='cik', suffixes=(f"{join_year_left}", f"{join_year_right}"))
# merged_df = pd.merge(merged_df, right_df, on='cik', suffixes=(f"{join_year_left}", f""))
# merged_df.head()
# merged_df.drop(f"Unnamed: 0", axis=1, inplace=True)
# # merged_df.drop(f"Unnamed: 0{join_year_right}", axis=1, inplace=True)
# merged_df.drop(f"sic{join_year_right}", axis=1, inplace=True)
# merged_df.drop(f"form_type{join_year_right}", axis=1, inplace=True)
# merged_df.drop(f"company_name{join_year_right}", axis=1, inplace=True)
# # merged_df.drop(f"Unnamed: 0{join_year_left}", axis=1, inplace=True)
# merged_df.drop(f"sic{join_year_left}", axis=1, inplace=True)
# merged_df.drop(f"form_type{join_year_left}", axis=1, inplace=True)
# merged_df.drop(f"company_name{join_year_left}", axis=1, inplace=True)
# merged_df.head()
# merged_df = merged_df.rename(columns={"report_period_end_date": "report_period_end_date2012", "file_date": "file_date2012"})
# merged_df.head()
# years_in_data = list(range(2007, 2013))
# columns_to_rename = {f"{year_t}_statement": f"statement{year_t}" for year_t in years_in_data}
# print(f"{columns_to_rename=}")
# merged_df = merged_df.rename(columns=columns_to_rename)
# merged_df.head()

In [53]:
len(merged_df.cik.unique())

2832

In [54]:
# merged_df.to_csv("/data/workspace_files/lazy_price_replication/10k_statements_merged_2007_to_2012.csv")

Link to cusip

In [56]:
cik_to_cusip_mapping = pd.read_csv("/data/workspace_files/lazy_price_replication/cik_to_cusip_wrds.csv")

In [57]:
merged_with_cusip = pd.merge(cik_to_cusip_mapping, merged_df, on='cik')

In [60]:
merged_with_cusip.head()

,cik,cusip_full,cusip,report_period_end_date2007,file_date2007,statement2007,report_period_end_date2008,file_date2008,statement2008,report_period_end_date2009,...,statement2010,report_period_end_date2011,file_date2011,statement2011,company_name,sic,form_type,report_period_end_date2012,file_date2012,statement2012
0,1800,002824100,00282410,20071231,20080219,ITEM 7. MANAGEMENTS DISCUSSION AND ANALYSIS OF...,20081231,20090220,ITEM 7. MANAGEMENTS DISCUSSION AND ANALYSIS OF...,20091231,...,ITEM 7. MANAGEMENTS DISCUSSION AND ANALYSIS OF...,20111231,20120221,ITEM 7. MANAGEMENTS DISCUSSION AND ANALYSIS OF...,ABBOTT LABORATORIES,2834,10-K,20121231,20130215,ITEM 7. MANAGEMENTS DISCUSSION AND ANALYSIS OF...
1,1800,002824100,00282410,20071231,20080219,ITEM 7. MANAGEMENTS DISCUSSION AND ANALYSIS OF...,20081231,20090220,ITEM 7. MANAGEMENTS DISCUSSION AND ANALYSIS OF...,20091231,...,ITEM 7. MANAGEMENTS DISCUSSION AND ANALYSIS OF...,20111231,20120221,ITEM 7. MANAGEMENTS DISCUSSION AND ANALYSIS OF...,ABBOTT LABORATORIES,2834,10-K/A,20121231,20130327,ITEM 7. MANAGEMENTS DISCUSSION AND ANALYSIS OF...
2,2034,004446100,00444610,20070630,20070910,ITEM 7. MANAGEMENTS DISCUSSION AND ANALYSIS OF...,20080630,20080905,Item 7. Managements Discussion and Analysis of...,20090630,...,Item 7. Managements Discussion and Analysis of...,20110630,20110909,Item 7. Managements Discussion and Analysis of...,ACETO CORP,5160,10-K,20120630,20120907,Item 7. Managements Discussion and Analysis of...
3,2178,006351308,00635130,20071231,20080331,Item 7. MANAGEMENTS DISCUSSION AND ANALYSIS OF...,20081231,20090324,Item 7. MANAGEMENTS DISCUSSION AND ANALYSIS OF...,20091231,...,Item 7. MANAGEMENTS DISCUSSION AND ANALYSIS OF...,20111231,20120323,Item 7. MANAGEMENTS DISCUSSION AND ANALYSIS OF...,"ADAMS RESOURCES & ENERGY, INC.",5172,10-K,20121231,20130315,Item 7. MANAGEMENTS DISCUSSION AND ANALYSIS OF...
4,2186,005320205,00532020,20071231,20080305,Item 7. Managements Discussion and Analysis of...,20081231,20090309,Item 7. Managements Discussion and Analysis of...,20091231,...,Item 7. Managements Discussion and Analysis of...,20111231,20120306,ITEM 7. MANAGEMENTS DISCUSSION AND ANALYSIS OF...,RELM WIRELESS CORP,3663,10-K,20121231,20130305,ITEM 7. MANAGEMENTS DISCUSSION AND ANALYSIS OF...


In [59]:
merged_with_cusip.drop(f"Unnamed: 0", axis=1, inplace=True)

In [94]:
len(merged_with_cusip)

8456

In [99]:
len(merged_with_cusip.cik.unique())

2683

In [101]:
len(merged_with_cusip.cusip.unique())

4367

In [95]:
len(merged_df)

4707

In [98]:
len(merged_df.cik.unique())

2832

In [100]:
len(merged_with_cusip)

8456

In [61]:
# merged_with_cusip.to_csv("/data/workspace_files/lazy_price_replication/10k_statements_merged_2007_to_2012.csv")

In [1]:
import wrds

In [2]:
db = wrds.Connection()

Enter your WRDS username [datalore]: jaya_gaur


KeyboardInterrupt: KeyboardInterrupt: Interrupted by user

In [74]:
db.describe_table(library="crsp_a_stock", table="wrds_names_query")

Approximately 181358 rows in crsp_a_stock.wrds_names_query.


,name,nullable,type,comment
0,permno,True,INTEGER,PERMNO
1,hdrprimaryexch,True,VARCHAR(1),Header Primary Exchange
2,nasdissuno,True,INTEGER,Nasdaq Issue Number
3,hdrsiccd,True,INTEGER,Header SIC Code
4,permco,True,INTEGER,PERMCO
5,secinfostartdt,True,DATE,Security Information Start Date
6,secinfoenddt,True,DATE,Security Information End Date
7,securitybegdt,True,DATE,Begin Date of Stock Data
8,securityenddt,True,DATE,End Date of Stock Data
9,cusip,True,VARCHAR(8),CUSIP


In [92]:
def get_cusip_to_ticker_mapping():
    cusip_str = [f"'{cuip}'" for cuip in merged_with_cusip.cusip.unique()]
    all_cusip = ",".join(cusip_str)
    print(f"{len(cusip_str)=}")
    query = f"SELECT cusip, ticker from crsp_a_stock.wrds_names_query WHERE cusip in ({all_cusip}) group by cusip, ticker"
    query_res = db.raw_sql(query)
    output_filename = f"/data/workspace_files/lazy_price_replication/cusip_to_ticker.csv"
    print(f"about to save query result to: {output_filename}")
    query_res.to_csv(output_filename)

In [93]:
get_cusip_to_ticker_mapping()

len(cusip_str)=4367
about to save query result to: /data/workspace_files/lazy_price_replication/cusip_to_ticker.csv


In [4]:
cusip_to_ticker = pd.read_csv("/data/workspace_files/lazy_price_replication/cusip_to_ticker.csv")

### Data Processing to link cik to cusip to ticker name

1. There are duplicate entries in the original merged 10k file. We first de-dup it by group by cik and choose the first item in each group
2. Next we want to link 10k file from cik to cusip (needed to further link ticker name). Since the mapping from cusip to cik can be many to many, we only consider those that have unique mapping from cusip to cik
3. once we have populated the cusip in the original 10k file, simply merge it with the cusip to ticker data frame to get the corresponding ticker. NOTE: since each cusip can be associated with multiple tickers, to make it easier, we only consider the cusip that has unique ticker mapping
4. finally, since yahoo finance does not have data for all the stocks, we filter the data set on the stock prices that yahoo finance has

In [49]:
# 1
merged_data_2007_to_2012_after_group = merged_data_2007_to_2012.groupby('cik', as_index=False).first()

In [20]:
# 2
cik_to_cusip_mapping = pd.read_csv("/data/workspace_files/lazy_price_replication/cik_to_cusip_wrds.csv")
cusip_per_count = cik_to_cusip_mapping.groupby("cusip").count()
cusip_per_count_one = cusip_per_count[cusip_per_count['cik'] == 1]
cik_to_cusip_mapping_unique_cusip = cik_to_cusip_mapping[cik_to_cusip_mapping["cusip"].isin(cusip_per_count_one.index)]
print(f"{len(cik_to_cusip_mapping_unique_cusip)}, {len(cik_to_cusip_mapping)}")
print(f"{len(cik_to_cusip_mapping_unique_cusip.cik.unique())}")
cik_per_count = cik_to_cusip_mapping_unique_cusip.groupby("cik").count()
cik_per_count_one = cik_per_count[cik_per_count['cusip'] == 1]
cik_to_cusip_mapping_unique_cusip_unique_cik = cik_to_cusip_mapping_unique_cusip[cik_to_cusip_mapping_unique_cusip["cik"].isin(cik_per_count_one.index)]
print(f"{len(cik_to_cusip_mapping_unique_cusip_unique_cik)}, {len(cik_to_cusip_mapping)}")

In [50]:
merged_data_2007_to_2012_after_group_with_cusip = pd.merge(merged_data_2007_to_2012_after_group, cik_to_cusip_mapping_unique_cusip_unique_cik, on='cik')

In [53]:
# 3
cusip_to_ticker_mapping = pd.read_csv("/data/workspace_files/lazy_price_replication/cusip_to_ticker.csv")

In [58]:
cusip_to_ticker_per_count_unique = cusip_to_ticker_per_count[cusip_to_ticker_per_count["ticker"] == 1]

In [59]:
cusip_to_ticker_mapping_one_to_one = cusip_to_ticker_mapping[cusip_to_ticker_mapping["cusip"].isin(cusip_to_ticker_per_count_unique.index)]

In [61]:
merged_data_2007_to_2012_after_group_with_cusip_and_ticker = pd.merge(merged_data_2007_to_2012_after_group_with_cusip, cusip_to_ticker_mapping_one_to_one, on='cusip')

In [68]:
merged_data_2007_to_2012_after_group_with_cusip_and_ticker.to_csv("/data/workspace_files/lazy_price_replication/10k_full_with_ticker_name.csv")

In [28]:
# 4, note this was done after we download the yahoo finance data, so the dataframe name changed
merged_data_2007_to_2012_filtered_by_yahoo = merged_data_2007_to_2012[~merged_data_2007_to_2012["ticker"].isin(tickers_with_empty_df)]

In [31]:
merged_data_2007_to_2012_filtered_by_yahoo.to_csv("/data/workspace_files/lazy_price_replication/10k_final.csv")

In [6]:
duplicates_specific = cusip_to_ticker.duplicated(subset=['cusip'], keep=False)

In [7]:
duplicate_rows_specific = cusip_to_ticker[duplicates_specific]

In [8]:
unique_rows_specific = cusip_to_ticker[~duplicates_specific]

In [10]:
duplicates_specific_full = merged_data_2007_to_2012.duplicated(subset=['cik'], keep=False)

In [11]:
unique_rows_specific_full = merged_data_2007_to_2012[~duplicates_specific_full]